In [64]:
import pandas as pd
from anndata import AnnData, read_h5ad
import scanpy as sc
from scanpy import read
from pyspark.mllib.linalg.distributed import CoordinateMatrix, MatrixEntry
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler, PCA

In [3]:
adata_gene_train = sc.read("/home/asmauger/biostat625final/rna_train_hvg_scaled.h5ad")
adata_gene_test = sc.read("/home/asmauger/biostat625final/rna_test_hvg_scaled.h5ad")
adata_protein_train = sc.read("/home/asmauger/biostat625final/prot_train_scaled.h5ad")
adata_protein_test = sc.read("/home/asmauger/biostat625final/prot_test_scaled.h5ad")

In [4]:
spgtrain = adata_gene_train.X
spgtest = adata_gene_test.X
spptrain = adata_protein_train.X
spptest = adata_protein_test.X
spgtrain_meta = pd.concat([adata_gene_train.obs['day'],adata_gene_train.obs['donor'],adata_gene_train.obs['cell_type']],axis=1)
spgtest_meta = pd.concat([adata_gene_test.obs['day'],adata_gene_test.obs['donor'],adata_gene_test.obs['cell_type']],axis=1)

In [5]:
# set executor memory + new spark session
SparkContext.setSystemProperty('spark.executor.memory', '20g')
SparkContext.setSystemProperty('spark.driver.memory', '20g')
SparkContext.setSystemProperty('spark.driver.maxResultSize', '5g')
spark = SparkSession.builder.appName('Spark new session').getOrCreate()
sparkContext=spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/17 12:09:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/17 12:09:57 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [7]:
# create the spark dataframe
sdf_gtrain = spark.createDataFrame(pd.DataFrame(spgtrain))
sdf_ptrain = spark.createDataFrame(pd.DataFrame(spptrain))

In [22]:
sdf_gtrain_meta = spark.createDataFrame(pd.DataFrame(spgtrain_meta))
sdf_gtest_meta = spark.createDataFrame(pd.DataFrame(spgtest_meta))

In [ ]:
sdf_gtest = spark.createDataFrame(pd.DataFrame(spgtest))
sdf_ptest = spark.createDataFrame(pd.DataFrame(spptest))

In [54]:
sdf_gtrain_combined = sdf_gtrain.join(sdf_gtrain_meta)
for i in sdf_ptrain.columns:
    sdf_ptrain=sdf_ptrain.withColumnRenamed(i,str("P"+i))
sdf_gtrain_combined_temp = sdf_gtrain_combined.join(sdf_ptrain.select('P1'))

In [66]:
ignore = ['P1', 'donor', 'cell_type'] #need to create dummy variables on donor and cell types
featureassembler = VectorAssembler(inputCols = [i for i in sdf_gtrain_combined_temp.columns if i not in ignore], 
                                   outputCol = "explanatory variables")
sdf_gtrain_combined_temp = featureassembler.transform(sdf_gtrain_combined_temp)

In [68]:
pca = PCA(k=30, inputCol="explanatory variables")
pca.setOutputCol("pca_features")
model = pca.fit(output)

22/12/17 15:17:36 ERROR Inbox: An error happened while processing message in the inbox for LocalSchedulerBackendEndpoint
java.lang.OutOfMemoryError: Java heap space


Exception in thread "dispatcher-event-loop-2" java.lang.OutOfMemoryError: Java heap space
ERROR:root:KeyboardInterrupt while sending command.][Stage 12:>   (0 + 0) / 1]
Traceback (most recent call last):
  File "/home/dqiaoran/.conda/envs/spark_env/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/dqiaoran/.conda/envs/spark_env/lib/python3.8/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/home/dqiaoran/.conda/envs/spark_env/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [69]:
# when finish running spark, run this line to end the spark session (it frees memory)
spark.stop

<bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x2b2072cd4e50>>

In [61]:
fd_train = output.select('explanatory variables', 'P1')
regressor = LinearRegression(featuresCol = 'explanatory variables', labelCol = 'P1', maxIter=10, regParam=0.3, elasticNetParam=0.8)
regressor = regressor.fit(fd_train)

In [ ]:
tlgtrain = list(zip(list(spgtrain.nonzero()[0]), list(spgtrain.nonzero()[1]), list(spgtrain.data)))
tlgtest = list(zip(list(spgtest.nonzero()[0]), list(spgtest.nonzero()[1]), list(spgtest.data)))
tlptrain = list(zip(list(spptrain.nonzero()[0]), list(spptrain.nonzero()[1]), list(spptrain.data)))
tlptest = list(zip(list(spptest.nonzero()[0]), list(spptest.nonzero()[1]), list(spptest.data)))

# create RDD object
coor_gtrain = sparkContext.parallelize(tlgtrain, numSlices = 500)
coor_gtest = sparkContext.parallelize(tlgtest, numSlices = 500)
coor_ptrain = sparkContext.parallelize(tlptrain, numSlices = 500)
coor_ptest = sparkContext.parallelize(tlptest, numSlices = 500)

cm_gtrain = CoordinateMatrix(coor_gtrain.map(lambda coords: MatrixEntry(*coords)))
cm_gtest = CoordinateMatrix(coor_gtest.map(lambda coords: MatrixEntry(*coords)))
cm_ptrain = CoordinateMatrix(coor_ptrain.map(lambda coords: MatrixEntry(*coords)))
cm_ptest = CoordinateMatrix(coor_ptest.map(lambda coords: MatrixEntry(*coords)))